In [1]:
#!pip install pandas matplotlib seaborn spotipy

In [9]:
# ********** Import Statements ************************************************************ #
import sys
import spotipy
import spotipy.util as util

import pandas as pd
import math
import json

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 500)

In [2]:
# Spotify user IDs for testing: these are public information.
user_ids = {'Cassie':'1242091675',
            'Caity':'12166088827',
            'Ben':'yaojxcyh5tczzlln0ygm9sfla',
            'Alex':'12130992491',
            'Jason':'jason.albert'}

In [13]:
class Sameify:
    
    def __init__(self, user_id):
        self.id = user_id
        self.client_info = json.load(open('/Users/cassielebauer/Documents/Projects/secret credentials/sameify_creds.json'))
        self.client_id = self.client_info['client_id']
        self.client_secret = self.client_info['client_secret']
        
        # Connect to Spotify
        self.sp = self.spotify_connect()
        self.playlists = self.get_all_playlists() 
        #self.playlist_tracks = self.get_all_playlist_tracks(self.playlists['playlist_id'][0])
        
        
    # Spotify credentials
    scope = 'playlist-modify-public user-library-read user-read-recently-played'
    #client_id removed
    #client_secret removed
    redirect_uri = 'http://localhost:1410/'

    # Connects to Spotify
    def spotify_connect(self):
        token = util.prompt_for_user_token(self.id,
                                           self.scope,
                                           self.client_id,
                                           self.client_secret,
                                           self.redirect_uri)

        sp = spotipy.Spotify(auth=token)
        return sp
    
    #def overstep_limit(self, item_id):

    def get_playlist_id(self, playlist_name):
        return self.playlists.loc[self.playlists['playlist_name'] == playlist_name,'playlist_id'].values[0]
    
    def get_playlist_name(self, playlist_id):
        return self.playlists.loc[self.playlists['playlist_id'] == playlist_id,'playlist_name'].values[0]
    
    def playlist_tracks(self, playlist_id):
        ret_df = pd.DataFrame()
        
        playlist = self.sp.user_playlist_tracks(self.id, playlist_id)
        playlist_name = self.get_playlist_name(playlist_id)
        tracks = playlist['items']
        
        while playlist['next']:
            playlist = self.sp.next(playlist)
            tracks.extend(playlist['items'])
            
        for track in tracks:
            x = track['track']
            data = {'playlist_id'  : playlist_id,
                    'playlist_name': playlist_name,
                    'track_name'   : x['name'],
                    'track_id'     : x['id'], 
                    'track_artist_id' : x['artists'][0]['id'],
                    'track_artist' : x['artists'][0]['name'], #[artist['name'] for artist in x['artists']]
                    'added_by'     : track['added_by']['id'],
                    'added_at'     : track['added_at']
                   }
            data_df = pd.DataFrame([data])
            song_deets = pd.DataFrame(self.sp.audio_features(data['track_id']))
            
            full_data = pd.concat([data_df, song_deets],axis=1)
            
            ret_df = ret_df.append(full_data)
        ret_df.reset_index(drop=True, inplace=True)
        return ret_df
            
#     def get_all_recently_played(self):
#         tracks = pd.DataFrame()
        
#         total = self.sp.user_playlists(self.id)['total'] #number of playlists
#         limit = 50 # pull at a time
        
#         for x in range(0, total, limit): # for each set of playlists
#             new_playlist = self.sp.user_playlists(self.id, limit=limit, offset=x)
#             for x in new_playlist['items']:
#                 items = {'playlist_name': x['name'],
#                          'playlist_id'  : x['id'],
#                          'owner_name'   : x['owner']['display_name'],
#                          'owner_id'     : x['owner']['id'],
#                          'num_tracks'   : x['tracks']['total'],
#                          'collaborative': x['collaborative'],
                         
                         
#         Cassie.sp.current_user_recently_played()

    def get_all_playlists(self):
        playlists = pd.DataFrame()
        
        total = self.sp.user_playlists(self.id)['total'] #number of playlists
        limit = 50 # pull at a time
        
        for x in range(0, total, limit): # for each set of playlists
            new_playlist = self.sp.user_playlists(self.id, limit=limit, offset=x)
            for x in new_playlist['items']:
                items = {'playlist_name': x['name'],
                         'playlist_id'  : x['id'],
                         'owner_name'   : x['owner']['display_name'],
                         'owner_id'     : x['owner']['id'],
                         'num_tracks'   : x['tracks']['total'],
                         'collaborative': x['collaborative'],
                         }
                playlists = playlists.append(pd.DataFrame([items]))
        playlists.reset_index(drop=True, inplace=True)
        return playlists

In [14]:
def recursive_skim(obj, d={}, f=0):
    for x in obj.keys():
        level = obj[x]
        print(f, '\t'*f, '*', x, type(level), '[0] {}'.format(type(level[0] if type(level) == list else level)) if type(level) == list else '')
        if type(level) in (list, dict):
            level_next = level[0] if type(level) == list else level
            if type(level_next) in (list, dict):
                recursive_skim(level_next, d, f+1)

In [15]:
Cassie = Sameify(user_ids['Cassie'])

In [17]:
Cassie.playlists

playlist_name             playlist_id   owner_name  \
0    All Songs (as of 04/05/21)  2RtUO6fWeIZBfjb0rAS9MW  Cassie Beth   
1          Retreating Into Calm  4NXxuPwKPkOStlrdEZsKxX  Cassie Beth   
2             Crossowew Archive  48jPm106dfQk2OAEvqP8td  Cassie Beth   
3            The Nechromanticon  49VEjhySMHWupyOsZPBYfG  Cassie Beth   
4    Calling All Theatre Nerds™  3etppTqdEy4kdUPXd3PMTi  Cassie Beth   
5               s e r e n a d e  06tWibmjNAkzRPoJHIIojc  Cassie Beth   
6         Maria Mena's Growthhh  1cpRCsHIU3qQOGa3StaqmG  Cassie Beth   
7                  For Ken's CD  5I9MF3IKlkJOdSMfNDP7uF  Cassie Beth   
8                            😏🤗  5Zmh3EF4xA2HiTH9hHBOMR  Cassie Beth   
9                            👑😌  4XULke3Hi4JvXu6i8T5urV  Cassie Beth   
10              CD Time Capsule  6YgBM46RmNRhT8pD4NuKWg  Cassie Beth   
11                   Peak Feels  40mwrRCP1P2ERL7Gwa2fy9  Cassie Beth   
12               Breathe me in.  6malBbojU8s6wUt5Yj4JqN  Cassie Beth   
13              Inside my head.  4l4PP6JDfqNeMpKoSf0vIa  Cassie Beth   
14               Mind on Repeat  5nZIG3UYY49QvQPH2gHopz  Cassie Beth   
15               Teenage Cassie  6Jz70BHkApBxmXdaN0o0zP  Cassie Beth   
16                  Summer 2017  71rzcr9mf3hBTqKa3c75vT  Cassie Beth   
17                  p e r n i l  6fatl3RODT1jh0HZRh4qsN  Cassie Beth   
18             Indiciiooooss 🧐🎶  6EwpIC3NLeemswSWEbDADy  Cassie Beth   
19           💃 Baila Al Ritmo 💃  1uBliNHYvDYxssRr7guPRS  Cassie Beth   
20           Best of All Worlds  65U8CnzLQICVeuvxTUH7ZB  Cassie Beth   
21                  SIIII YESSS  1r6ySloJL2slFo7PpjYOHM  Cassie Beth   
22                  האהובים עלי  2mhVQi9TBDg6650iX8tkzx  Cassie Beth   
23                        日本の音楽  396IBmgnUwnJCIuOMFqjcG  Cassie Beth   
24               Linguistic Mix  794KUKGn3bYJwkbnpfdRng  Cassie Beth   
25                    Sign Away  3FqfZiA67MK7GOCKcU6PwM  Cassie Beth   
26         Mes Favoris Français  14y9LLwP37Om9GC8g588r5  Cassie Beth   
27                          ITG  3LRk3HSmJ6g4sCAXr5yZbl  Cassie Beth   
28                W o r k o u t  6T98XwlsUS18zHdwrQzNx9  Cassie Beth   
29      If I Ever Did Burlesque  2sXA2Ds2b7bXZ7w4ivEcUa  Cassie Beth   
30                    s0ngz4ddr  22MWDMuqPszgUV2GP4lK7b  Cassie Beth   
31   Crochelliptical Experience  1j9SDMbodOp04G8DIZ9pGf  Cassie Beth   
32             My Heart is Open  0BAJvhMqj26oRa0L35D1kl  Cassie Beth   
33                  Spoken Word  1oNDlD7o9IUfiFzV6wLqR3  Cassie Beth   
34           FORGOT THE NAME AH  6NP9vcDFTL7RVK8TpikVvW  Cassie Beth   
35              SKIP / DEAD AIR  3V5OvrzC1ZFuUAkAMWj5rG  Cassie Beth   
36     I Wish I Knew These Then  6p8y7nLj82MSReFuPcScIy  Cassie Beth   
37             High School Band  2xxqGjadUK2bR3vQVRbPx9  Cassie Beth   
38       I'm in a good mood. <3  5PWFdfVhoruxlB5KWHjLMW  Cassie Beth   
39                     Level Up  3CnBnAH4F36LKq3d3e6b0F  Cassie Beth   
40               So Much Energy  71hWUmKrSU6GVVYjGgHKq0  Cassie Beth   
41         Also 15 Minute Burst  77WmvOUOArCMwKowetnmbs  Cassie Beth   
42              15 Minute Burst  799Ugy5YmkTHj0M15q7yZL  Cassie Beth   
43                        Ouch.  6EK8BdhLybBUiULWPgByVR  Cassie Beth   
44                         Mood  5lJjzeR1G7qMtqG9qp8kfY  Cassie Beth   
45                      Karaoke  0Z7c5zbkMmmdHOoX1H9i8L  Cassie Beth   
46            Perfect Harmonies  1Sc1UfKCURYymoMQ6WeScn  Cassie Beth   
47             Singing Playlist  4jXLpy8TZmj4QtsMAKpqoJ  Cassie Beth   
48           Cinematic Chillout  37i9dQZF1DWVFJtzvDHN4L      Spotify   
49      25 min Focus Session :)  4Xb7GXhiOlwczSmTYOdmvi  Cassie Beth   
50               Mongaup Valley  4pv3Stohe2IfKHdWMrfLZl  Cassie Beth   
51          From A Man Long Ago  7hSzUCWn09mTRsOhoSJhPk  Cassie Beth   
52           Discovered & Loved  7MpbX5HSu7FnsK8E4ZcGzD  Cassie Beth   
53             A girl long ago.  5bk0ZuIZyf7dUX7SWZJapE  Cassie Beth   
54                         test  19qJLyzv8oSBlzGwb5kFsV  C

In [18]:
Art = Sameify('artlolos1')

In [21]:
Art.sp.

{'country': 'US',
 'display_name': 'Cassie Beth',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'external_urls': {'spotify': 'https://open.spotify.com/user/1242091675'},
 'followers': {'href': None, 'total': 66},
 'href': 'https://api.spotify.com/v1/users/1242091675',
 'id': '1242091675',
 'images': [{'height': None,
   'url': 'https://scontent-dfw5-2.xx.fbcdn.net/v/t1.6435-1/p320x320/190974522_10218106581639340_7329445740732901937_n.jpg?_nc_cat=106&ccb=1-3&_nc_sid=0c64ff&_nc_ohc=5-851QUERVcAX9kk7Im&_nc_ht=scontent-dfw5-2.xx&tp=6&oh=7e3a94e8c1162e2df0f5dd7536d9c82b&oe=60E4FA5B',
   'width': None}],
 'product': 'premium',
 'type': 'user',
 'uri': 'spotify:user:1242091675'}

In [30]:
Alex = Sameify(user_ids['Alex'])

In [ ]:
Caity = Sameify(user_ids['Caity'])
Ben = Sameify(user_ids['Ben'])
Jason = Sameify(user_ids['Jason'])

# How to see playlists

In [ ]:
Cassie.get_all_playlists()

In [ ]:
playlists = Cassie.get_all_playlists()
playlists

# How to see songs in a playlist

In [ ]:
a_tracks = Alex.playlist_tracks(Alex.get_playlist_id('Liked from Radio'))
a_tracks

In [ ]:
a_tracks.iloc[0]

# See original depth tree of playlist info

In [ ]:
x = Alex.sp.user_playlist(Alex.id, Alex.get_playlist_id('Liked from Radio'))
recursive_skim(x)

# See new depth tree of playlist info
I pulled out the values I felt were most useful, see above in the class definition

In [ ]:
y = Alex.playlist_tracks(Alex.get_playlist_id('Liked from Radio'))
recursive_skim(y)

# Visualize a playlist

In [ ]:
# simplify and melt playlist
def melt_df(df):
    new_df = df[['playlist_name', 'track_id', 'acousticness', 'danceability', 'energy', 'liveness', 'valence', 'speechiness']]
    return new_df.melt(id_vars=['playlist_name', 'track_id'])

In [ ]:
# melt playlist
alex_playlist_melt = melt_df(a_tracks)
alex_playlist_melt

In [ ]:
# display musical information
fig, ax = plt.subplots(figsize=(20,15))
ax = sns.boxplot(x='variable', y='value', hue='playlist_name', data=alex_playlist_melt, palette='Accent')

# Compare playlists

In [ ]:
# pull french playlist from cassie
c_tracks = Cassie.playlist_tracks(Cassie.get_playlist_id('Inside my head.'))

# melt playlist
cassie_playlist_melt = melt_df(c_tracks)
cassie_playlist_melt

In [ ]:
melted = alex_playlist_melt.append(cassie_playlist_melt)

# display musical information
fig, ax = plt.subplots(figsize=(20,15))
ax = sns.boxplot(x='variable', y='value', hue='playlist_name', data=melted, palette='Accent')

# Play around with data

In [ ]:
len(Cassie.get_all_playlists())
#162

In [ ]:
p_id = Cassie.get_playlist_id('Miki Ratsula')


In [ ]:
Cassie.sp.

In [ ]:
Cassie.sp.user_playlist_add_tracks(user = Cassie, 
                                   playlist_id = p_id,
                                   tracks = ['6i4m9NMsIenenO2djGUTpe', '2OJFqxSN8Qk37TpzZy2UZz'],
                                   position = [1,3])

In [ ]:
ts = ['6i4m9NMsIenenO2djGUTpe', '2OJFqxSN8Qk37TpzZy2UZz']
ps = [1,3]

In [ ]:
p_id = Alex.get_playlist_id('Plane')

In [ ]:
xx = Cassie.sp.current_user_recently_played()

In [ ]:
xx

In [ ]:
xx['items'][0]['track']['type']

In [ ]:
xx['next']

In [ ]:
xx.keys()

In [ ]:
https://api.spotify.com/v1/me/player/recently-played?before=1581012036871&limit=50'

In [ ]:
xx['cursors']

In [ ]:
[x['track']['name'] for x in xx['items']]

In [8]:
p_id = Cassie.get_playlist_id('Miki Ratsula')


In [ ]:
Cassie.sp.search()

In [13]:
Cassie.sp.playlist_tracks(p_id)['items'][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [18]:
pd.to_datetime(Cassie.sp.playlist_tracks(p_id)['items'][0]['track']['album']['release_date']).year

2017

In [ ]:
Cassie.get_playlist

In [31]:
a_tracks = Alex.playlist_tracks(Alex.get_playlist_id('Liked from Radio'))

In [32]:
a_tracks = a_tracks[['track_name', 'track_id', 'track_artist_id', 'track_artist']].copy()

In [50]:
Cassie.get_all_playlists()

playlist_name  \
0                                          January '20   
1                                         Miki Ratsula   
2                                             Cleaning   
3                                    Perfect Harmonies   
4                                      Discover Weekly   
5                                        Release Radar   
6                                                  New   
7                                          Alarm songs   
8                                                    0   
9                                                    1   
10                                    Morning Playlist   
11                                        December '19   
12                                        November '19   
13                                         October '19   
14                                       September '19   
15                                          August '19   
16                                            July '19   
17                                            June '19   
18                                             May '19   
19                                           April '19   
20                                           March '19   
21                                        February '19   
22                                         January '19   
23                                        December '18   
24                                        November '18   
25                                         October '18   
26                                       September '18   
27                                          August '18   
28                                            July '18   
29                                          SoundHound   
30                               Release Radar Archive   
31                             Discover Weekly Archive   
32                                       Test Playlist   
33                                Inspired By Warriors   
34                                      Breathe me in.   
35                                     Inside my head.   
36                                      Mind on Repeat   
37                                     Past Obsessions   
38                                      Teenage Cassie   
39                                         Summer 2017   
40                                Zero’s Pet Playlist    
41                                            Hamilton   
42                                   Beats to think to   
43                                               Sleep   
44           **DEEP FOCUS** Orchestral Post-Minimalism   
45                                          Deep Focus   
46                                  Cinematic Chillout   
47                                 Cinematic Classical   
48                                        Soundwave CD   
49                                         האהובים עלי   
50                                    Morning Acoustic   
51                                          Chill Lah!   
52                                 PTX Christmas Songs   
53                                  Acapella Christmas   
54                                         Happy Beats   
55                                      It's ALT Good!   
56                                        Mood Booster   
57                                        Hot Rhythmic   
58                                   Best of - Remixes   
59                                        Mega Hit Mix   
60                                             Workout   
61                                    Headphone Tester   
62                                                Rawr   
63                                      Skip skip skip   
64                            I Wish I Knew These Then   
65                                  FORGOT THE NAME AH   
66                                 Your Top Songs 2018   
67                                                 🤘🏻🐰   
68                                   Ri

In [36]:
a_tracks['genre'] = Alex.playlist_tracks(Alex.get_playlist_id('Liked from Radio'))['track_artist_id'].apply(lambda x: Alex.sp.artist(x)['genres'])

In [51]:
[print(Alex.sp.artist(x)['genres']) for x in Cassie.playlist_tracks(Cassie.get_playlist_id('Inside my head.'))['track_artist_id']] 


['candy pop', 'neo mellow', 'norwegian pop', 'norwegian singer-songwriter']
['alternative rock', 'baroque pop', 'canadian indie', 'chamber pop', 'indie folk', 'indie pop', 'indie rock', 'modern folk rock', 'modern rock', 'stomp and holler']
['marching band', 'wind ensemble']
['american contemporary classical', 'concert band']
['pop', 'pop rap']
['alternative pop rock', 'neon pop punk', 'pop emo', 'pop punk', 'texas pop punk']
[]
['a cappella']
[]
['a cappella']
['modern ska punk', 'pop punk', 'punk', 'ska', 'ska punk', 'skate punk']
['indiecoustica', 'stomp and holler']
['a cappella']
['acoustic pop', 'neo mellow', 'pop', 'pop rock', 'viral pop']
[]
['comic', 'geek rock', 'permanent wave', 'zolo']
['candy pop', 'neo mellow', 'norwegian pop', 'norwegian singer-songwriter']
['metropopolis', 'modern rock', 'pop', 'pop rock', 'rock', 'tropical house']
[]
['contemporary country', 'country', 'country road', 'modern country rock']
['piano rock']
['acoustic pop', 'ectofolk', 'indiecoustica', '

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
a_genres

In [15]:
recursive_skim(Cassie.sp.playlist_tracks(p_id))

0  * href <class 'str'> 
0  * items <class 'list'> [0] <class 'dict'>
1 	 * added_at <class 'str'> 
1 	 * added_by <class 'dict'> 
2 		 * external_urls <class 'dict'> 
3 			 * spotify <class 'str'> 
2 		 * href <class 'str'> 
2 		 * id <class 'str'> 
2 		 * type <class 'str'> 
2 		 * uri <class 'str'> 
1 	 * is_local <class 'bool'> 
1 	 * primary_color <class 'NoneType'> 
1 	 * track <class 'dict'> 
2 		 * album <class 'dict'> 
3 			 * album_type <class 'str'> 
3 			 * artists <class 'list'> [0] <class 'dict'>
4 				 * external_urls <class 'dict'> 
5 					 * spotify <class 'str'> 
4 				 * href <class 'str'> 
4 				 * id <class 'str'> 
4 				 * name <class 'str'> 
4 				 * type <class 'str'> 
4 				 * uri <class 'str'> 
3 			 * available_markets <class 'list'> [0] <class 'str'>
3 			 * external_urls <class 'dict'> 
4 				 * spotify <class 'str'> 
3 			 * href <class 'str'> 
3 			 * id <class 'str'> 
3 			 * images <class 'list'> [0] <class 'dict'>
4 				 * height <class 'int'> 
4 				 